<a href="https://colab.research.google.com/github/R-SIVA/Spam-Emails-Detection/blob/main/Spam_Mail_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('/content/spam - Copy.csv')
df.head(20)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
df.value_counts()

Category  Message                                                                                                                                                
ham       Sorry, I'll call later                                                                                                                                     30
          I cant pick the phone right now. Pls send a message                                                                                                        12
          Ok...                                                                                                                                                      10
          Okie                                                                                                                                                        4
          7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st "Ur Lovely Friendship"... good morning dear     4
                                                                                                                                                                     ..
          I was just callin to say hi. Take care bruv!                                                                                                                1
          I was gonna ask you lol but i think its at 7                                                                                                                1
          I was at bugis juz now wat... But now i'm walking home oredi... Ü so late then reply... I oso saw a top dat i like but din buy... Where r ü now?            1
          I was about to do it when i texted. I finished a long time ago and showered and er'ything!                                                                  1
spam      we tried to contact you re your response to our offer of a new nokia fone and camcorder hit reply or call 08000930705 for delivery                          1
Name: count, Length: 5157, dtype: int64

In [6]:
df[df['Category'] == 'spam'].value_counts()

Category  Message                                                                                                                                                           
spam      Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed £1000 cash or £5000 prize!                   4
          #ERROR!                                                                                                                                                               3
          Loan for any purpose £500 - £75,000. Homeowners + Tenants welcome. Have you been previously refused? We can still help. Call Free 0800 1956669 or text back 'help'    3
          Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days.                                                            3
          December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906         3
                                                                                                                                                                               ..
          Here is your discount code RP176781. To stop further messages reply stop. www.regalportfolio.co.uk. Customer Services 08717205546                                     1
          Hey Boys. Want hot XXX pics sent direct 2 ur phone? Txt PORN to 69855, 24Hrs free and then just 50p per day. To stop text STOPBCM SF WC1N3XX                          1
          Hey I am really horny want to chat or see me naked text hot to 69698 text charged at 150pm to unsubscribe text stop 69698                                             1
          Hi I'm sue. I am 20 years old and work as a lapdancer. I love sex. Text me live - I'm i my bedroom now. text SUE to 89555. By TextOperator G2 1DA 150ppmsg 18+        1
          we tried to contact you re your response to our offer of a new nokia fone and camcorder hit reply or call 08000930705 for delivery                                    1
Name: count, Length: 641, dtype: int64

In [7]:
df[df['Category'] == 'ham'].value_counts()

Category  Message                                                                                                                                 
ham       Sorry, I'll call later                                                                                                                      30
          I cant pick the phone right now. Pls send a message                                                                                         12
          Ok...                                                                                                                                       10
          Okie                                                                                                                                         4
          Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..     4
                                                                                                                                                      ..
          I dont know ask to my brother. Nothing problem some thing that. Just i told .                                                                1
          I dont know exactly could you ask chechi.                                                                                                    1
          I dont know oh. Hopefully this month.                                                                                                        1
          I dont know what to do to come out of this so only am ask questions like this dont mistake me.                                               1
          … we r stayin here an extra week, back next wed. How did we do in the rugby this weekend? Hi to and and , c u soon "                         1
Name: count, Length: 4516, dtype: int64

In [8]:
# Making a 'spam' column and replacing spam with 1
df['spam'] = df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [9]:
#droping the category column
df.drop('Category',inplace =True, axis =1)

In [10]:
x= df.Message
y= df['spam']

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [12]:
clf = Pipeline([('vectorizer', CountVectorizer()),('nb', MultinomialNB())])

In [13]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [14]:
clf.score(x_train,y_train)

0.9930446488669509

In [15]:
clf.score(x_test,y_test)

0.9883408071748879

In [16]:
testing_emails = ['hi, wanna hangout at 10? i heard there is a good chinese restaurant nearby. see you soon'
                 ,"don't miss this chance to win 100$ dollars"]

In [17]:
clf.predict(testing_emails)

array([0, 1])